<a href="https://colab.research.google.com/github/shreyasomkuwar26/multi-agent-investment-advisor/blob/main/Multi_Agent_Investment_Advisor_Finnhub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comprehensive Investment Advisory Workflow Using CrewAI


In this notebook, we illustrate a sophisticated investment advisory system built using **CrewAI**. The workflow combines multiple AI agents, each equipped with specialized tools, to:
1. Gather financial data and news about a company.
2. Analyze the data comprehensively.
3. Provide actionable investment recommendations.

---
### Workflow Overview
1. Define custom tools to retrieve financial data, income statements, and stock prices.
2. Create agents for data gathering, analysis, and recommendation tasks.
3. Define tasks aligned with these agents' roles and objectives.
4. Configure a crew to execute these tasks sequentially, leveraging collaboration between agents.
5. Run the workflow and observe the results.

---
### Code Walkthrough
Below is the implementation.


### Step 1: Import Required Libraries and Declare in-built Tools

In [ ]:
## initial setup for Colab.
## if you are using this in local jupyter notebook, skip this cell
!pip install crewai
!pip install crewai-tools
!pip install duckduckgo-search
!pip install finnhub-python

In [ ]:
from crewai import Agent, Task
from crewai.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from datetime import datetime

# Current date for context
Now = datetime.now()
Today = Now.strftime("%d-%b-%Y")

# Define a web search tool
@tool("DuckDuckGo Search")
def search_tool(search_query: str):
    """Search the internet for information on a given topic"""
    return DuckDuckGoSearchRun().run(search_query)

### Step 2: Define Custom Tools

In [ ]:
# Simple cache implementation
cache = {}
cache_expiry = {}

def get_cached_data(key, expiry_seconds=300):
    """Get data from cache if not expired."""
    if key in cache and time.time() < cache_expiry.get(key, 0):
        return cache[key]
    return None

def set_cached_data(key, data, expiry_seconds=300):
    """Set data in cache with expiry time."""
    cache[key] = data
    cache_expiry[key] = time.time() + expiry_seconds

In [ ]:
from crewai.tools import tool
import finnhub
import json
import time
import os

from google.colab import userdata

# Get the API key from userdata
FINNHUB_API_KEY = userdata.get('finnhub_api_key')

# Now you can use FINNHUB_API_KEY to initialize the Finnhub client
finnhub_client = finnhub.Client(api_key=FINNHUB_API_KEY)


@tool("Get current stock price")
def get_current_stock_price(symbol: str) -> str:
    """Use this function to get the current stock price for a given symbol.

    Args:
        symbol (str): The stock symbol. For Indian stocks, use format like 'RELIANCE.NS' or 'TCS.BO'.

    Returns:
        str: The current stock price or error message.
    """
    """Use this function to get the current stock price..."""
    cache_key = f"price_{symbol}"
    cached_result = get_cached_data(cache_key, 60)  # Cache for 60 seconds
    if cached_result:
        return cached_result

    try:
        # Add a small delay to avoid rate limiting
        time.sleep(0.5)

        # Handle Indian stock exchange symbols
        api_symbol = symbol
        if '.NSE' in symbol or '.BSE' in symbol:
            # Finnhub requires different format for Indian stocks
            # Strip the .NSE or .BSE and add the exchange info
            base_symbol = symbol.split('.')[0]
            exchange = symbol.split('.')[1]
            # For some Indian exchanges, you might need to modify the symbol format
            api_symbol = f"{base_symbol}:{exchange}"

        # Get the quote
        quote = finnhub_client.quote(api_symbol)

        if quote and 'c' in quote:
            current_price = quote['c']
            set_cached_data(cache_key, str(current_price))  # Cache the result
            return f"{current_price:.2f}"
        else:
            return f"Could not fetch current price for {symbol}"
    except Exception as e:
        return f"Error fetching current price for {symbol}: {e}"

@tool
def get_company_info(symbol: str):
    """Use this function to get company information and current financial snapshot for a given stock symbol.

    Args:
        symbol (str): The stock symbol. For Indian stocks, use format like 'RELIANCE.NS' or 'TCS.BO'.

    Returns:
        JSON containing company profile and current financial snapshot.
    """
    try:
        # Add a small delay to avoid rate limiting
        time.sleep(0.5)

        api_symbol = symbol
        if '.NSE' in symbol or '.BSE' in symbol:
            # Finnhub requires different format for Indian stocks
            # Strip the .NSE or .BSE and add the exchange info
            base_symbol = symbol.split('.')[0]
            exchange = symbol.split('.')[1]
            # For some Indian exchanges, you might need to modify the symbol format
            api_symbol = f"{base_symbol}:{exchange}"

        # Get company profile
        profile = finnhub_client.company_profile2(symbol=api_symbol)

        # Get quote data
        quote = finnhub_client.quote(api_symbol)

        # Get basic financials
        financials = finnhub_client.company_basic_financials(api_symbol, 'all')

        # Create a cleaned info dictionary
        company_info_cleaned = {
            "Name": profile.get("name"),
            "Symbol": profile.get("ticker"),
            "Current Stock Price": f"{quote.get('c')} {profile.get('currency', 'USD')}",
            "Market Cap": profile.get("marketCapitalization"),
            "Sector": profile.get("finnhubIndustry"),
            "Industry": profile.get("finnhubIndustry"),
            "Country": profile.get("country"),
            "Exchange": profile.get("exchange"),
            "IPO": profile.get("ipo"),
            "Logo": profile.get("logo"),
            "Website": profile.get("weburl"),
        }

        # Add financial metrics if available
        if financials and 'metric' in financials:
            metrics = financials['metric']
            company_info_cleaned.update({
                "52 Week Low": metrics.get("52WeekLow"),
                "52 Week High": metrics.get("52WeekHigh"),
                "P/E Ratio": metrics.get("peBasicExclExtraTTM"),
                "EPS": metrics.get("epsBasicExclExtraItemsTTM"),
                "Dividend Yield": metrics.get("dividendYieldIndicatedAnnual"),
                "ROE": metrics.get("roeTTM"),
                "ROA": metrics.get("roaTTM"),
                "Debt to Equity": metrics.get("totalDebtOverTotalEquityQuarterly"),
                "Revenue Growth": metrics.get("revenueGrowthTTMYoy"),
            })

        return json.dumps(company_info_cleaned)
    except Exception as e:
        return f"Error fetching company profile for {symbol}: {e}"

@tool
def get_income_statements(symbol: str):
    """Use this function to get income statements for a given stock symbol.

    Args:
        symbol (str): The stock symbol. For Indian stocks, use format like 'RELIANCE.NS' or 'TCS.BO'.

    Returns:
        JSON containing income statements or an empty dictionary.
    """
    try:
        # Add a small delay to avoid rate limiting
        time.sleep(0.5)

        # Handle Indian stock exchange symbols
        api_symbol = symbol
        if '.NSE' in symbol or '.BSE' in symbol:
            # Finnhub requires different format for Indian stocks
            # Strip the .NSE or .BSE and add the exchange info
            base_symbol = symbol.split('.')[0]
            exchange = symbol.split('.')[1]
            # For some Indian exchanges, you might need to modify the symbol format
            api_symbol = f"{base_symbol}:{exchange}"

        # Get financial statements
        financials = finnhub_client.financials_reported(symbol=api_symbol, freq='annual')

        # Extract income statements if available
        if financials and 'data' in financials and len(financials['data']) > 0:
            income_statements = []
            for report in financials['data']:
                if 'report' in report and 'ic' in report['report']:
                    income_statements.append({
                        'year': report.get('year'),
                        'quarter': report.get('quarter'),
                        'income_statement': report['report']['ic']
                    })
            return json.dumps(income_statements)
        else:
            return f"No income statements found for {symbol}"
    except Exception as e:
        return f"Error fetching income statements for {symbol}: {e}"

### Step 3: Define the Agents



In [ ]:
from crewai import Agent

# Agent for gathering company news and information
news_info_explorer = Agent(
    role='News and Info Researcher',
    goal='Gather and provide the latest news and information about a company from the internet',
    #llm='gpt-4o',
    llm='gpt-4.1-2025-04-14',
    verbose=True,
    backstory=(
        'You are an expert researcher, who can gather detailed information about a company. '
        'Consider you are on: ' + Today
    ),
    tools=[search_tool],
    cache=True,
    max_iter=5,
)

# Agent for gathering financial data
data_explorer = Agent(
    role='Data Researcher',
    goal='Gather and provide financial data and company information about a stock',
    #llm='gpt-4o',
    llm='gpt-4.1-2025-04-14',
    verbose=True,
    backstory=(
        'You are an expert researcher, who can gather detailed information about a company or stock. '
        'For Indian stocks, use format like "RELIANCE.NSE" for NSE or "TCS.BSE" for BSE. '
        'Intelligently figure out it is an Indian or US stock'
        'For US stocks, just use the ticker symbol like "AAPL". '
        'Use Indian units for numbers (lakh, crore) for Indian stocks only. '
        'Consider you are on: ' + Today
    ),
    tools=[get_company_info, get_income_statements],
    cache=True,
    max_iter=5,
)

# Agent for analyzing data
analyst = Agent(
    role='Data Analyst',
    goal='Consolidate financial data, stock information, and provide a summary',
    #llm='gpt-4o',
    llm='gpt-4.1-2025-04-14',
    verbose=True,
    backstory=(
        'You are an expert in analyzing financial data, stock/company-related current information, and '
        'making a comprehensive analysis. Use Indian units for numbers (lakh, crore). '
        'Consider you are on: ' + Today
    ),
)

# Agent for financial recommendations
fin_expert = Agent(
    role='Financial Expert',
    goal='Considering financial analysis of a stock, make investment recommendations',
    #llm='gpt-4o',
    llm='gpt-4.1-2025-04-14',
    verbose=True,
    tools=[get_current_stock_price],
    max_iter=5,
    backstory=(
        'You are an expert financial advisor who can provide investment recommendations. '
        'Consider the financial analysis, current information about the company, current stock price, '
        'and make recommendations about whether to buy/hold/sell a stock along with reasons. '
        'For Indian stocks, use format like "RELIANCE.NSE" for NSE or "TCS.BSE" for BSE. '
        'For US stocks, just use the ticker symbol like "AAPL". '
        'Intelligently figure out it is an Indian or US stock. '
        'Use Indian units for numbers (lakh, crore) for Indian stocks only. '
        'Consider you are on: ' + Today
    ),
)

### Step 4: Define the Tasks

In [ ]:
from crewai import Task

# Task to gather financial data of a stock
get_company_financials = Task(
    description="Get key financial data for stock: {stock}. Focus on the most important metrics only.",
    expected_output="Summary of key financial metrics for {stock}. Keep it concise and under 1000 words.",
    agent=data_explorer,
)

# Task to gather company news
get_company_news = Task(
    description="Get latest 3-5 important news items about company: {stock}",
    expected_output="Brief summary of 3-5 latest significant news items. Keep it under 800 words.",
    agent=news_info_explorer,
)

# Task to analyze financial data and news
analyse = Task(
    description="Analyze the financial data and news, focusing on the most important points.",
    expected_output="Concise analysis of the stock's key metrics and news impact. Maximum 1500 words.",
    agent=analyst,
    context=[get_company_financials, get_company_news],
    output_file='Analysis.md',
)


# Task to provide financial advice
advise = Task(
    description="Make a recommendation about investing in a stock, based on analysis provided and current stock price. "
                "Explain the reasons.",
    expected_output="Recommendation (Buy / Hold / Sell) of a stock backed with reasons elaborated."
                    "Response in Mark down format.",
    agent=fin_expert,
    context=[analyse],
    output_file='Recommendation.md',
)

### Step 5: Set Up the Crew

In [ ]:
from crewai import Crew, Process
from datetime import datetime

# Callback function to print a timestamp
def timestamp(Input):
    print(datetime.now())

# Define the crew with agents and tasks in sequential process
crew = Crew(
    agents=[data_explorer, news_info_explorer, analyst, fin_expert],
    tasks=[get_company_financials, get_company_news, analyse, advise],
    verbose=True,
    Process=Process.sequential,
    step_callback=timestamp,
)

### Step 5: Run the Crew and Observe Results

In [ ]:
# Set your OpenAI API key or any other LLM API key
from google.colab import userdata
import os

# Set your Groq API key or any other LLM API key
os.environ['OPENAI_API_KEY'] = userdata.get('openai_key')

# Run the crew with a specific stock
result = crew.kickoff(inputs={'stock': 'TSLA'})

# Print the final result
print("Final Result:", result)


---
### Conclusion
This workflow demonstrates a detailed investment advisory process using multiple agents and tasks. The system showcases the integration of custom tools and collaborative agents in CrewAI to provide actionable financial insights.
